In [2]:
import qmps
import xmps
import cirq
from xmps import iMPS
import numpy as np
def ap(A, _abs = True):
    if _abs:
        print(np.around(np.abs(A), 2))
    else:
        print(np.around(A, 2))

In [3]:
# Initial State |ψ> = |0101010...>
# transfer matrix looks like ;
# A[σ, i, j] = [σᵃ, σᵇ] where σᵃ & σᵇ = |0X1| & |1X0| respectively 

A = np.random.randn(2,2,2)
A[0,:,:] = np.array([[0,1], [0,0]])
A[1,:,:] = np.array([[0,0], [1,0]])

from qmps.tools import tensor_to_unitary, environment_to_unitary
from qmps.represent import get_env_exact 
U = tensor_to_unitary(A)
U

array([[0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.]])

In [4]:
V = 1/np.sqrt(2)*np.array([[1,1,0,0],[0,0,1,1],[0,0,1,-1],[1,-1,0,0]])
V

array([[ 0.70710678,  0.70710678,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.70710678,  0.70710678],
       [ 0.        ,  0.        ,  0.70710678, -0.70710678],
       [ 0.70710678, -0.70710678,  0.        ,  0.        ]])

In [5]:
# Check that this analytical result hold in the simulator
from qmps.represent import Tensor

def simulate_state(gates, qubits, _print = False):
    qbs = cirq.LineQubit.range(qubits)
    circuit = cirq.Circuit.from_ops([gate(*qbs[-2:]) if i == 0 else gate(*qbs[-2-i:-i]) for i, gate in enumerate(gates)])
    
    if _print:
        print(circuit)
        
    sim = cirq.Simulator()
    res = sim.simulate(circuit)
    density_matrix = res.density_matrix_of(qbs[0:1])
    return density_matrix

state = Tensor(U, "U")
env = Tensor(V, "V")

gates = [env, state]
ρ_full = simulate_state(gates, 3, True)
print("-------------------------------------")
gate = [env]
ρ = simulate_state(gate, 2, True)
print("-------------------------------------")
print(ρ_full, "\n\n", ρ)

0: ───────U───
          │
1: ───V───U───
      │
2: ───V───────
-------------------------------------
0: ───V───
      │
1: ───V───
-------------------------------------
[[0.49999997+0.j 0.        +0.j]
 [0.        +0.j 0.49999997+0.j]] 

 [[0.49999997+0.j 0.        +0.j]
 [0.        +0.j 0.49999997+0.j]]


In [6]:
# Build the Hamiltonian for scar state evolution:
from scipy.linalg import expm

P = np.array([[1,0],[0,0]])
X = np.array([[0,1],[1,0]])
H = np.kron(np.kron(P,X),P)

dt = 0.1
W = Tensor(expm(-1j * H * dt), "W")



In [7]:
def put_env_on_left_site(q, ret_n=False):
    # Take a matrix q (2x2) and create U such that 
    # (right 0-|---|--0
    #          | u |        =  q_{ij}
    # (left) i-|---|--j 
    q = q.T
    a, b, c, d = q.reshape(-1)
    n = np.sqrt(np.abs(a)**2+ np.abs(c)**2+ np.abs(b)**2+ np.abs(d)**2)
    guess = np.array([[a, c.conj(), b, d.conj()], [c, -a.conj(), d, -b.conj()]])/n
    orth = null_space(guess).conj().T
    A = np.concatenate([guess, orth], axis=0)
    A = cirq.unitary(cirq.SWAP)@A
    if ret_n:
        return A, n
    else:
        return A
def get_env_off_left_site(A):
    z = np.array([1, 0])
    return np.tensordot(np.tensordot(A.reshape(2, 2, 2, 2), z, [3, 0]), z, [1, 0]).T

def put_env_on_right_site(q, ret_n=False):
    q = q
    a, b, c, d = q.reshape(-1)
    n = np.sqrt(np.abs(a)**2+ np.abs(c)**2+ np.abs(b)**2+ np.abs(d)**2)
    guess = np.array([[a, b, d.conj(), -c.conj()], [c, d, -b.conj(), a.conj()]])/n
    orth = null_space(guess).conj().T
    A = np.concatenate([guess, orth], axis=0)
    #A = cirq.unitary(cirq.SWAP)@A
    if ret_n:
        return A, n
    else:
        return A

def get_env_off_right_site(A):
    z = np.array([1, 0])
    return np.tensordot(np.tensordot(A.reshape(2, 2, 2, 2), z, [2, 0]), z, [0, 0])



In [21]:
from qmps.represent import FullStateTensor, FullEnvironment, get_env_exact, Environment
from qmps.tools import unitary_to_tensor, environment_from_unitary, tensor_to_unitary
from xmps.spin import U4
from xmps.iMPS import Map, iMPS
from qmps.time_evolve_tools import merge, put_env_on_left_site, put_env_on_right_site

def sim_overlap(params, A, W):
    
    U_ = FullStateTensor(U4(params))
    A_ = unitary_to_tensor(U4(params))
    U = FullStateTensor(tensor_to_unitary(A))
    
    _, r = Map(A,A_).right_fixed_point()
    
    R = Environment(put_env_on_left_site(r), 'θR')
    left = put_env_on_right_site(r.conj().T)
    L = Environment(left, 'θL')

    q = cirq.LineQubit.range(7)

    circuit = cirq.Circuit.from_ops([
        cirq.H(q[4]),
        cirq.CNOT(q[4], q[5]),
        U(*q[3:5]),
        U(*q[2:4]),
        U(*q[1:3]),
        L(*q[0:2]),
        R(*q[5:7]),
        W(*q[2:5]),
        cirq.inverse(U_)(*q[1:3]),
        cirq.inverse(U_)(*q[2:4]),
        cirq.inverse(U_)(*q[3:5]),
        cirq.CNOT(q[4], q[5]),
        cirq.H(q[4])
        ])

    sim = cirq.Simulator()
    X =sim.simulate(circuit).final_state
    return X 

def scar_obj_fun(params, A, W):
    return np.abs(sim_overlap(params, A, W)[0])*2    


params = np.random.randn(15)
U = U4(params)
A = iMPS().random(2,2).left_canonicalise()[0]
W = Tensor(np.eye(8), "W")

scar_obj_fun(params, A, W)





0.23958638310432434

In [23]:
from numpy.linalg import svd

def S(ρ):
    # S = Σᵢρᵢlog(ρᵢ) where ρᵢ = |λᵢ|^2 and λᵢ = singular values 
    u, s, v = svd(ρ) 
    s = np.square(np.abs(s)) 
    return np.dot(np.log2(s), s)
    

In [2]:
!git add -A
!git commit -m'changes in Many |Body Scars and some printed example circuits'
!git -pu

# On branch master
# Changes not staged for commit:
#   (use "git add/rm <file>..." to update what will be committed)
#   (use "git checkout -- <file>..." to discard changes in working directory)
#
#	modified:   Many_body_scars.ipynb
#	modified:   Time Evo.ipynb
#	deleted:    qmps/States.py
#	modified:   qmps/ground_state.py
#	modified:   qmps/new_time_evolve.py
#	modified:   qmps/qiskit_qmps.py
#	modified:   qmps/represent.py
#	modified:   qmps/rotosolve.py
#	modified:   qmps/time_evolve_tools.py
#	modified:   qmps/tools.py
#
# Untracked files:
#   (use "git add <file>..." to include in what will be committed)
#
#	E_op.txt
#	output.txt
#	represent_swap.txt
#	time_evolve.txt
no changes added to commit (use "git add" and/or "git commit -a")
